# Práctica 2 - Análisis numérico

## Autores:
<ul>
<li> López Espíndola Luis Enrique
<li> Mucio Alvarez Santos
<li> Muñiz Lescale Marco Aurelio
<li> Reynoso Sánchez Arturo Yitzack
</ul>

Modulos Necesarios:

In [ ]:
import numpy as np

Excepciones: Definimos nuestra clase de excepciones. Por las diferentes cosas raras que puedan salir,
se dan posibles errores previamente identificados y sino se lanza el error y se da como no clasificado



In [ ]:
class sslException(Exception):
    # Se inicia la clase determinando cual es la etiqueta del erorr
    def __init__(self, valor):
        self.valor = str(valor)

    # Se determina el cjto de errores previamente identificados (PI), se toma 
    # la etiqueta que inicio la clase, y se determina si esta en el cjto PI, si
    # si se lanza el mensaje correspondiente a esa excepccion, sino se
    # "catalogo" como un error no clasificado
    def __str__(self):
        errores = {"0": "Las dimensiones son diferentes. La matriz debe ser cuadrada.",
                   "1": "Los elementos de la diagonal son cero. La matriz no es invertible.",
                   "2": "Los elementos de la diagonal son cero.",
                   "3": "Los elementos de la diagonal deben ser positivos."}
        if self.valor in errores:
            return f"Error {self.valor}. {errores[self.valor]}"
        else:
            return "Error no clasificado"


Para facilitar un poco los codigos importamos un par de funciones hechas en clase con Jorge, con sus
respectivos comentarios

Sistema Triangular Inferior (Sustitucion hacia a adelante)\:

Definimos la rutina que realizará la sustitucion hacia adelante


In [ ]:
def STI(L, rhs):
    m, n = L.shape  # Dimensiones de L
    if m != n:  # En caso de que no sea cuadrada
        raise sslException(0)
    b = rhs.copy()
    # Creamos el vector, un vector basura del tamaño que se necesita
    x = np.empty((n,), dtype="float64")
    # Aqui empezamos a construir la sutitucion hacia adelante
    for j in range(n):
        if L[j, j] == 0:  # La diagonal tiene ceros, lo que nos impide invertirla
            raise sslException(1)
        x[j] = b[j]/L[j, j]
        for i in range(j+1, n):
            b[i] = b[i] - L[i, j]*x[j]
    return x

Sistema Triangular Superior (Sustitución hacia a atrás)

In [ ]:
def STS(U, rhs):
    m, n = U.shape  # Dimensiones de U
    if m != n:  # Si la matriz no es cuadrada
        raise sslException(0)
    b = rhs.copy()
    # Creamos el vector, un vector basura del tamaño que se necesita
    x = np.zeros(b.shape, dtype="float64")
    # Aqui empezamos a construir la sutitucion hacia atras
    # Aqui esta la clave para que vaya de Xn a X1
    for j in reversed(range(n)):
        if U[j, j] == 0:  # La diagonal tiene ceros, U no es invertible
            raise sslException(1)
        else:
            x[j] = b[j]/U[j, j]
            for i in range(j):
                b[i] = b[i] - U[i, j]*x[j]
    return x



# <FONT COLOR="blue">Ejercicio 25</FONT>

Dado el sistem $Ax = b$

<li><i> Resolver con eliminacion Gaussiana para
\begin{equation}
    A =
    \begin{pmatrix}
    2 & 4 & -2  \\
    4 & 9 & -3  \\
    -2 & -1 & 7 \\
    \end{pmatrix}\;\;\;
    b= \begin{pmatrix}
    2\\
    8\\
    10
    \end{pmatrix}.
    \end{equation}


In [ ]:
# Definimos nuestra funcion de eliminacion gaussiana
def Gauss(M, rhs):
    A, b = M.copy(), rhs.copy()  # Definimos nuestra matriz y solucion b, con lo recibido
    m, n = A.shape  # Dimensiones de A
    if m != n:  # Checa si la matriz es cuadrada
        raise sslException(0)
    # Se empieza la reduccion
    for j in range(n-1):
        if A[j, j] == 0:  # Ceros en la diagonal, A es no invertible
            raise sslException(2)
        for k in range(j+1, n):
            # Factor que permite se forme el cero en la entrada que necesitamos
            l = -A[k, j]/A[j, j]
            # Nuevo renglon
            for i in range(j+1, n):
                A[k, i] = A[k, i] + l*A[j, i]
            # Nueva entrada de la ultima columna de la matriz aumentada
            b[k] = b[k] + l*b[j]
    # Una vez reducido el jale, aplicamos la solucion a sistemas de ecuaciones y lo regresamos
    return STS(A, b)


Una vez definida la función podemos aplicar el método

In [ ]:
# Introducimos nuestras matrices
A = np.array([[2, 4, -2], [4, 9, -3], [-2, -1, 7]])
b = np.array([2, 8, 10])
print(f"A = \n{A}", f"b = \n{b}", sep="\n\n")

A = 
[[ 2  4 -2]
 [ 4  9 -3]
 [-2 -1  7]]

b = 
[ 2  8 10]


Resolvemos 

In [ ]:
x = Gauss(A, b)
print("Aplicando elimininacion Gaussiana tenemos que:",
      f"x = {x}",
      f"Comprobando con el modulo numpy, nuetra respuesta es: {np.allclose(A@x,b)}", 
      sep="\n")


Aplicando elimininacion Gaussiana tenemos que:
x = [-7.  4.  0.]
Comprobando con el modulo numpy, nuetra respuesta es: True


<li><i> Usar Factorizacion LU de A para resolver el sistema Ay = c, donde:
\begin{equation}
c= \begin{pmatrix}
    4\\
    8\\
    -6
    \end{pmatrix}.
\end{equation}

In [ ]:
c = np.array([4, 8, -6])

Definimos nuestra funcion de Factorizacion LU

In [ ]:
def lu(M):
    m, n = M.shape  # Dimensiones de M
    if m != n:  # Por si no es cuadrada
        raise sslException(0)
    # Se define U como copia de M y L una matriz identidad de n*n
    U, L = M.copy(), np.eye(n)
    # Aplicamos reduccion gaussiana para construir U
    # & L se construye con los elementos que se utlizan para hacer ceros debajo de la columna
    for j in range(n-1):
        if U[j, j] == 0:  # Si tiene ceros en la diagonal no es invertible
            raise sslException(2)
        for k in range(j+1, n):
            L[k, j] = U[k, j]/U[j, j]
            for i in range(j+1, n):
                U[k, i] = U[k, i] - L[k, j]*U[j, i]
        U[j+1:, j] = 0
    return L, U

Aplicamos la factorización LU

In [ ]:
L, U = lu(A)
print("Aplicandola factorizacion LU tenemos que:",
      f"L = \n{L}", f"U = \n{U}", f"Entoces \nLU = \n{L@U}",
      f"Comprobando con el modulo numpy, nuetra respuesta es: {np.allclose(L@U,A)}", sep="\n\n")

Aplicandola factorizacion LU tenemos que:

L = 
[[ 1.  0.  0.]
 [ 2.  1.  0.]
 [-1.  3.  1.]]

U = 
[[ 2  4 -2]
 [ 0  1  1]
 [ 0  0  2]]

Entoces 
LU = 
[[ 2.  4. -2.]
 [ 4.  9. -3.]
 [-2. -1.  7.]]

Comprobando con el modulo numpy, nuetra respuesta es: True


Y recordemos que $$LU\hat{y}=c \longleftrightarrow L\hat{z} = c y U\hat{y} = z$$
Entonces resolviendo el sistema tenemos que:

In [ ]:
z = STI(L, c) # Se resuelve el sistema de la triangular inferior
y = STS(U, z) # Se resuelve el sistema de la traingular superior 
print("Resolviendo el sistema tenemos que:",
      f"Lz = c -> z = {z}", f"& Uy = z -> y = {y}", sep="\n")


Resolviendo el sistema tenemos que:
Lz = c -> z = [  4.  16. -50.]
& Uy = z -> y = [-1.39814815 -4.03703704  1.85185185]


# <FONT COLOR="blue">Ejercicio 26</FONT>

Sean $A$ y $b$ dados como sigue:
<ol type = "a"> 
<li> <i>) 

\begin{equation}
A =
    \begin{pmatrix}
    4 & -1 & 3  \\
    -8 & 4 & -7  \\
    12 & 1 & 8 \\
    \end{pmatrix}\;\;\;
    b= \begin{pmatrix}
    -8\\
    19\\
    -19
    \end{pmatrix}.
    \end{equation}

<li> <i>) 
\begin{equation}
A =
    \begin{pmatrix}
    1 & 4 & -2 & 1  \\
    -2 & -4 & -3 & 1 \\
    1 & 16 & -17 & 9 \\
    2 & 4 & -9 & -3 
    \end{pmatrix}\;\;\;
    b= \begin{pmatrix}
    3.5\\
    -2.5\\
    15\\
    10.5
  \end{pmatrix}.
\end{equation}

<li> <i>) 
\begin{equation}
A =
    \begin{pmatrix}
    4 & 5 & -1 & 2 & 3  \\
    12 & 13 & 0 & 10 & 3 \\
    -8 & -8 & 5 & -11 & 4 \\
    16 & 18 & -7 & 20 & 4 \\
    -4 & -9 & 31 & -31 & -1
    \end{pmatrix}\;\;\;
    b= \begin{pmatrix}
    34\\
    93\\
    -33\\
    131\\
    -58
  \end{pmatrix}.
\end{equation}

Resolver cada sistema $A\hat{x} = \hat{b}$ con las rutinas:
<ol type = "b"> 
<li><i> Factorizacion LU.
<li><i> Factorizacion LU con pivoteo parcial
<li><i>Factorizacion LU con pivoteo total

Comparar los resultados

Empezamos definiendo las funciones de nuestras rutinas
<ol type= "*">

*   Para la Factorización LU se ocupara la runia lu(M), definada en el ejercicio 25
*   Definimos la rutina lupp(M) para el pivoteo parcial 

In [ ]:

def lupp(M):
    m, n = M.shape  # Dimensiones de M
    if m != n:  # Por si no es cuadrada
        raise sslException(0)
    # Se define U como copia de M y L una matriz identidad de n*n
    U, L= np.array(M,dtype = "float64") , np.zeros(M.shape)
    piv = np.arange(0,n)
    # Aplicamos reduccion gaussiana para construir U
    # & L se construye con los elementos que se utlizan para hacer ceros debajo del 
    # pivote
    for j in range(n-1):
        # pivote
        p = np.argmax(abs(U[piv[j:],j]))+ j 
        if U[piv[p], j] == 0: 
            raise sslException(4)
        if piv[p] != piv[j]:
            z = piv[p]
            piv[p] = piv[j]
            piv[j] = z
        for k in range(j+1, n):
            L[piv[k], j] = U[piv[k], j]/U[piv[j], j]
            for i in range(j+1, n):
                U[piv[k], i] = U[piv[k], i] - L[piv[k], j]*U[piv[j], i]
        U[piv[j+1:], j] = 0
    for l in range(n):
        L[piv[l], l] = 1
    return L, U[piv,:], piv

La rutina de factorizacion LU con pivote Total no podimos obtnerla, desarrollamos lo siguiente pero presenta detalles:

In [ ]:
def lupt(M):
    m, n = M.shape  # Dimensiones de M
    if m != n:  # Por si no es cuadrada
        raise sslException(0)
    # Se define U como copia de M y L una matriz identidad de n*n
    U, L= np.array(M,dtype = "float64") , np.zeros(M.shape)
    pivr = np.arange(0,n)
    pivc = np.arange(0,n)
    # Aplicamos reduccion gaussiana para construir U
    # & L se construye con los elementos que se utlizan para hacer ceros debajo de la columna
    for j in range(n-1):
        # pivote
        u = U[pivr[j:],pivc[j:]]
        p = np.unravel_index(np.argmax(abs(u)),U.shape)
        if U[pivr[p[0]], pivc[p[1]]] == 0: 
            raise sslException(2)
        if pivr[p[0]] != pivr[j]:
            z = pivr[p[0]]
            pivr[p[0]] = piv[j]
            pivr[j] = z
        if pivc[p[1]] != pivc[j]:
            z = pivc[p[1]]
            pivc[p[1]] = pivc[j]
            pivc[j] = z
        for k in range(j+1, n):
            L[pivr[k], pivc[j]] = U[pivr[k], pivc[j]]/U[pivr[j], pivc[j]]
            for i in range(j+1, n):
                U[pivr[k], pivc[i]] = U[pivr[k], pivc[i]] - L[pivr[k], pivc[j]]*U[pivr[j], piv
        U[pivr[j+1:], pivc[j]] = 0
    for l in (range(n)):
        L[pivr[l], pivc[l]] = 1
    return L[pivr,:], U[:,pivc], pivr


Aplicamos nuestras rutinas a cada inciso 

a)

In [ ]:
A = np.array([[4,-1,3],[-8,4,7],[12,1,8]])
b = np.array([-8,19,-19])
# Aplicamos la factorizacion LU
L, U = lu(A)
print("Aplicandola factorizacion LU tenemos que:",
      f"L = \n{L}", f"U = \n{U}", f"Entoces \nLU = \n{L@U}",
      f"Comprobando con el modulo numpy, nuetra respuesta es: {np.allclose(L@U,A)}", sep="\n\n")

# Y recordemos que LUx = b sii  Ly = b & Ux = y
y = STI(L, b) # Se resuelve el sistema de la triangular inferior
x = STS(U, y) # Se resuelve el sistema de la traingular superior 

# Resolviendo el sistema tenemos que:
print("Resolviendo el sistema tenemos que:",
      f"Ly = c -> y = {y}", f"& Ux = y -> x = {x}", sep="\n")

# Aplicamos la factorizacion LU con pivoteo parcial
L, U, piv = lupp(A)
print("\nAplicandola factorizacion LU con pivoteo parcial tenemos que:",
      f"L = \n{L}", f"U = \n{U}", f"Entoces \nLU = \n{L@U}", 
      f"piv = \n{piv}",
      f"Comprobando con el modulo numpy, nuetra respuesta es: {np.allclose(L@U,A)}", sep="\n\n")

# Y recordemos que LUx = b sii  Ly = b & Ux = y
y = STI(L[piv,:], b) # Se resuelve el sistema de la triangular inferior
x = STS(U, y) # Se resuelve el sistema de la traingular superior 

# Resolviendo el sistema tenemos que:
print("Resolviendo el sistema tenemos que:",
      f"Ly = c -> y = {y}", f"& Ux = y -> x = {x}", sep="\n")

Aplicandola factorizacion LU tenemos que:

L = 
[[ 1.  0.  0.]
 [-2.  1.  0.]
 [ 3.  2.  1.]]

U = 
[[  4  -1   3]
 [  0   2  13]
 [  0   0 -27]]

Entoces 
LU = 
[[ 4. -1.  3.]
 [-8.  4.  7.]
 [12.  1.  8.]]

Comprobando con el modulo numpy, nuetra respuesta es: True
Resolviendo el sistema tenemos que:
Ly = c -> y = [-8.  3. -1.]
& Ux = y -> x = [-1.71296296  1.25925926  0.03703704]

Aplicandola factorizacion LU con pivoteo parcial tenemos que:

L = 
[[ 0.33333333 -0.28571429  1.        ]
 [-0.66666667  1.          0.        ]
 [ 1.          0.          0.        ]]

U = 
[[12.          1.          8.        ]
 [ 0.          4.66666667 12.33333333]
 [ 0.          0.          3.85714286]]

Entoces 
LU = 
[[ 4. -1.  3.]
 [-8.  4.  7.]
 [12.  1.  8.]]

piv = 
[2 1 0]

Comprobando con el modulo numpy, nuetra respuesta es: True
Resolviendo el sistema tenemos que:
Ly = c -> y = [ -8.  13. -12.]
& Ux = y -> x = [ 0.49007937 11.00793651 -3.11111111]


b)

In [ ]:
A = np.array([[1,4,-2,1],[-2,-4,-3,1],[1,16,-17,9],[2,4,-9,-3]])
b = np.array([3.5,-2.5,15,10.5])
# Aplicamos la factorizacion LU
L, U = lu(A)
print("Aplicandola factorizacion LU tenemos que:",
      f"L = \n{L}", f"U = \n{U}", f"Entoces \nLU = \n{L@U}",
      f"Comprobando con el modulo numpy, nuetra respuesta es: {np.allclose(L@U,A)}", sep="\n\n")

# Y recordemos que LUx = b sii  Ly = b & Ux = y
y = STI(L, b) # Se resuelve el sistema de la triangular inferior
x = STS(U, y) # Se resuelve el sistema de la traingular superior 

# Resolviendo el sistema tenemos que:
print("Resolviendo el sistema tenemos que:",
      f"Ly = c -> y = {y}", f"& Ux = y -> x = {x}", sep="\n")

# Aplicamos la factorizacion LU con pivoteo parcial
L, U, piv = lupp(A)
print("\nAplicandola factorizacion LU con pivoteo parcial tenemos que:",
      f"L = \n{L}", f"U = \n{U}", f"Entoces \nLU = \n{L@U}", 
      f"piv = \n{piv}",
      f"Comprobando con el modulo numpy, nuetra respuesta es: {np.allclose(L@U,A)}", sep="\n\n")

# Y recordemos que LUx = b sii  Ly = b & Ux = y
y = STI(L[piv,:], b) # Se resuelve el sistema de la triangular inferior
x = STS(U, y) # Se resuelve el sistema de la traingular superior 

# Resolviendo el sistema tenemos que:
print("Resolviendo el sistema tenemos que:",
      f"Ly = c -> y = {y}", f"& Ux = y -> x = {x}", sep="\n")


Aplicandola factorizacion LU tenemos que:

L = 
[[ 1.  0.  0.  0.]
 [-2.  1.  0.  0.]
 [ 1.  3.  1.  0.]
 [ 2. -1. -2.  1.]]

U = 
[[ 1  4 -2  1]
 [ 0  4 -7  3]
 [ 0  0  6 -1]
 [ 0  0  0 -4]]

Entoces 
LU = 
[[  1.   4.  -2.   1.]
 [ -2.  -4.  -3.   1.]
 [  1.  16. -17.   9.]
 [  2.   4.  -9.  -3.]]

Comprobando con el modulo numpy, nuetra respuesta es: True
Resolviendo el sistema tenemos que:
Ly = c -> y = [ 3.5  4.5 -2.   4. ]
& Ux = y -> x = [-0.5  1.  -0.5 -1. ]

Aplicandola factorizacion LU con pivoteo parcial tenemos que:

L = 
[[-0.5         0.14285714  0.07142857  1.        ]
 [ 1.          0.          0.          0.        ]
 [-0.5         1.          0.          0.        ]
 [-1.          0.          1.          0.        ]]

U = 
[[ -2.          -4.          -3.           1.        ]
 [  0.          14.         -18.5          9.5       ]
 [  0.           0.         -12.          -2.        ]
 [  0.           0.           0.           0.28571429]]

Entoces 
LU = 
[[  1.   4. 

c)


In [ ]:
A = np.array([[4,5,-1,2,3],[12,13,0,10,3],[-8,-8,5,-11,4],[16,18,-7,20,4],[-4,-9,31,-31,-1]])
b = np.array([34,93,-33,131,-58])
# Aplicamos la factorizacion LU
L, U = lu(A)
print("Aplicandola factorizacion LU tenemos que:",
      f"L = \n{L}", f"U = \n{U}", f"Entoces \nLU = \n{L@U}",
      f"Comprobando con el modulo numpy, nuetra respuesta es: {np.allclose(L@U,A)}", sep="\n\n")

# Y recordemos que LUx = b sii  Ly = b & Ux = y
y = STI(L, b) # Se resuelve el sistema de la triangular inferior
x = STS(U, y) # Se resuelve el sistema de la traingular superior 

# Resolviendo el sistema tenemos que:
print("Resolviendo el sistema tenemos que:",
      f"Ly = c -> y = {y}", f"& Ux = y -> x = {x}", sep="\n")

# Aplicamos la factorizacion LU con pivoteo parcial
L, U, piv = lupp(A)
print("\nAplicandola factorizacion LU con pivoteo parcial tenemos que:",
      f"L = \n{L}", f"U = \n{U}", f"Entoces \nLU = \n{L@U}", 
      f"piv = \n{piv}",
      f"Comprobando con el modulo numpy, nuetra respuesta es: {np.allclose(L@U,A)}", sep="\n\n")

# Y recordemos que LUx = b sii  Ly = b & Ux = y
y = STI(L[piv,:], b) # Se resuelve el sistema de la triangular inferior
x = STS(U, y) # Se resuelve el sistema de la traingular superior 

# Resolviendo el sistema tenemos que:
print("Resolviendo el sistema tenemos que:",
      f"Ly = c -> y = {y}", f"& Ux = y -> x = {x}", sep="\n")

Aplicandola factorizacion LU tenemos que:

L = 
[[ 1.  0.  0.  0.  0.]
 [ 3.  1.  0.  0.  0.]
 [-2. -1.  1.  0.  0.]
 [ 4.  1. -1.  1.  0.]
 [-1.  2.  4. -5.  1.]]

U = 
[[ 4  5 -1  2  3]
 [ 0 -2  3  4 -6]
 [ 0  0  6 -3  4]
 [ 0  0  0  5  2]
 [ 0  0  0  0  8]]

Entoces 
LU = 
[[  4.   5.  -1.   2.   3.]
 [ 12.  13.   0.  10.   3.]
 [ -8.  -8.   5. -11.   4.]
 [ 16.  18.  -7.  20.   4.]
 [ -4.  -9.  31. -31.  -1.]]

Comprobando con el modulo numpy, nuetra respuesta es: True
Resolviendo el sistema tenemos que:
Ly = c -> y = [34. -9. 26. 30. 40.]
& Ux = y -> x = [1. 2. 3. 4. 5.]

Aplicandola factorizacion LU con pivoteo parcial tenemos que:

L = 
[[ 0.25       -0.11111111  0.5         1.          0.        ]
 [ 0.75        0.11111111  0.25        0.16666667  1.        ]
 [-0.5        -0.22222222  1.          0.          0.        ]
 [ 1.          0.          0.          0.          0.        ]
 [-0.25        1.          0.          0.          0.        ]]

U = 
[[ 16.          18.       

En conclusión, tanto para el inciso a) como para el inciso b) notamos que el valor de y no cambia, en ninguna de las factorizaciones, mientras que el valor de la solución difiere de manera notable 

Aunque se incluyó la rutina LU con pivoteo, nos dimos cuenta que aun cuenta con un ligero error, el cual lastimosamente no hemos encontrado pero creimos importamte poner lo que hicimos y no dejarlo en blanco 

# <FONT COLOR="blue">Ejercicio 27</FONT>

# <FONT COLOR="blue">Ejercicio 28</FONT>

Dada la matriz 
$$A=
\begin{pmatrix}
1 & 0 & 0 & 0 & 1\\
-1 & 1 & 0 & 0 & 1\\
-1 & -1 & 1 & 0 & 1\\
-1 & -1 & -1 & 1 & 1\\
-1 & -1 & -1 & -1 & 1\\
\end{pmatrix}
$$
1. ¿Qué sucede cuando se usa Eliminación Gaussina con pivoteo parcial?.
2. Usar Eliminación Gaussina con pivoteo parcial para resolver sistemas de varios tamaños (al menos 5) con vectores b elegidos por cualquier criterio. Describir como se comporta la condición de la matriz.
3. Escribir una rutina que factorice la matriz (LU) con pivoteo total.


Con la eliminación Gaussiana del 

In [ ]:
# algoritmo para descomposicion A=LU
# la matriz A ya esta dada
def factorizacionLU(A):
    # dimension de la matriz
    n = len(A)
    # matriz L inicialmente es la identidad
    L = np.identity(n)
    # inicialmente la matriz A y la matriz U son iguales
    U = np.zeros((n,n))
    for i in range(0,n):
        for j in range(0,n):
            U[i][j] = A[i][j]
    # eliminacion gaussiana
    for i in range(0,n):
        for j in range(i+1,n):
            # guardar los factores de eliminacion gaussiana 
            # en la matriz L
            factor = U[j][i]/U[i][i]
            L[j][i] = factor
            # realizar eliminacion gaussiana en la matriz U
            # para quedar de forma triangular superior
            for k in range(i,n):
                U[j][k] = U[j][k] - factor*U[i][k]
    return L,U

def main():
    # matriz A
    A = np.array([[1,0,0,0,1],[-1,1,0,0,1],[-1,-1,1,0,1],[-1,-1,-1,1,1],[-1,-1,-1,-1,1]])
    # llamar funcion de descomposicion LU para obtener/llenar matrices L,U
    L,U=factorizacionLU(A)
    #se imprimen las matrices y el vector b
    print(A)
    print(L)
    print(U)
    
main()

[[ 1  0  0  0  1]
 [-1  1  0  0  1]
 [-1 -1  1  0  1]
 [-1 -1 -1  1  1]
 [-1 -1 -1 -1  1]]
[[ 1.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.]
 [-1. -1.  1.  0.  0.]
 [-1. -1. -1.  1.  0.]
 [-1. -1. -1. -1.  1.]]
[[ 1.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  2.]
 [ 0.  0.  1.  0.  4.]
 [ 0.  0.  0.  1.  8.]
 [ 0.  0.  0.  0. 16.]]


# <FONT COLOR="blue">Ejercicio 29</FONT>
Usar eliminación Gaussiana sin pivoteo para resolver el sistema:
$$
  \begin{bmatrix}
  \epsilon & 1 \\
  1        & 1
  \end{bmatrix}
  \begin{bmatrix}
  x_1 \\
  x_2
  \end{bmatrix}
  =
  \begin{bmatrix}
  1+\epsilon \\
  2
  \end{bmatrix}
$$
para $\epsilon = 10^{-2k}$, con $k = 1,...,10$. La solución exacta es $x = \begin{bmatrix} 1 & 1 \end{bmatrix}^t$, ¿cómo se comporta la solución conforme $\epsilon$ decrece?

In [ ]:
import numpy as np



# Construcción del Metodo Gauss sin pivoteo:
# Excepciones
class slsException(Exception):
    def __init__(self,valor):
        self.valor = str(valor)
    def __str__(self):
        errores = {"0":"Las dimensiones son diferentes. La matriz debe ser cuadrada.",
                  "1":"Los elementos de la diagonal son cero. La matriz no es invertible.",
                  "2":"Los elementos de la diagonal son cero.",
                  "3":"Los elementos de la diagonal deben ser positivos."}
        if self.valor in errores:
            return f"Error {self.valor}. {errores[self.valor]}"
        else:
            return "Error no clasificado"
# Sistema triangular superior
def STS(U,rhs):
    m,n = U.shape # dimensiones
    if m != n: # misma dimensión
        raise slsException(0)
    b = rhs.copy() # copia para no modificar el original
    x = np.zeros(b.shape,dtype = "float64") # almacena la solución
    for j in reversed(range(n)): # aplicamos la fórmula para sustituir hacia atrás
        if U[j,j] == 0:
            raise slsException(1)
        else:
            x[j] = b[j]/U[j,j]
            for i in range(j):
                b[i] = b[i] - U[i,j]*x[j] # aplicamos fórmula de sustitución
    return x
# Metodo de Gauss sin pivoteo
def Gauss(M,rhs):
    A,b = M.copy(),rhs.copy() # copias para no modificar los originales
    m,n = A.shape # dimensiones
    if m != n: # misma dimensión cuadrada
        raise slsException(0)
    for j in range(n-1): # sobre las columnas
        if A[j,j] == 0: # para si el pivotte es cero
            raise slsException(2)
        for k in range(j+1,n):
            l = -A[k,j]/A[j,j] # multiplicadores para formar ceros en las columnas
            for i in range(j+1,n): # siguiente fila
                A[k,i] = A[k,i] + l*A[j,i] # aplicamos transforación a la submatriz restante
            b[k] = b[k] + l*b[j]
    return STS(A,b) # solución



# Implementación del Metodo Gauss sin pivoteo:
for k in range(10,0,-1): # K = 10,9,...,1
  # Valor para epsilon
  e = 10**(-2*k)
  # Construccion de las matrices A y b dado epsilon
  A = np.array([[e,1],[1,1]])
  b = np.array([1+e,2])
  # Solución al sistema de ecuaciones
  x = Gauss(A,b)
  print("Para K =",k,"la solución es:",f"x = {x}",np.allclose(A@x,b))



# Observaciones:
print("\n Observamos que la solución es más exacta conforme epsilon decrece")

Para K = 10 la solución es: x = [0. 1.] False
Para K = 9 la solución es: x = [0. 1.] False
Para K = 8 la solución es: x = [2.22044605 1.        ] False
Para K = 7 la solución es: x = [0.99920072 1.        ] False
Para K = 6 la solución es: x = [0.99986686 1.        ] False
Para K = 5 la solución es: x = [1.00000008 1.        ] True
Para K = 4 la solución es: x = [0.99999999 1.        ] True
Para K = 3 la solución es: x = [1. 1.] True
Para K = 2 la solución es: x = [1. 1.] True
Para K = 1 la solución es: x = [1. 1.] True

 Observamos que la solución es más exacta conforme epsilon decrece


# <FONT COLOR="blue">Ejercicio 30</FONT>

# <FONT COLOR="blue">Ejercicio 31</FONT>

# <FONT COLOR="blue">Ejercicio 32</FONT>
Encontrar la factorización de Cholesky de $A$ para las siguientes matrices:
$$
  A =
  \begin{bmatrix}
  2  & -1 & 0 \\
  -1 & 2  & -1 \\
  0  & -1 & 2
  \end{bmatrix}; \hspace{1cm}
  B = 
  \begin{bmatrix}
  4 & 1  & 1  & 1 \\
  1 & 3  & -1 & 1 \\
  1 & -1 & 2  & 0 \\
  1 & 1  & 0  & 2
  \end{bmatrix}
  \\
  C =
  \begin{bmatrix}
  4  & 1  & -1 & 0 \\
  1  & 3  & -1 & 0 \\
  -1 & -1 & 5  & 2 \\
  0  & 0  & 2  & 4
  \end{bmatrix}; \hspace{1cm}
  E =
  \begin{bmatrix}
  6  & 2 & 1  & -1 \\
  2  & 4 & 1  & 0 \\
  1  & 1 & 4  & -1 \\
  -1 & 0 & -1 & 3
  \end{bmatrix}
$$
Utilizando:

(a) $A = LL^t$

(b) $A = \hat{L}\hat{D}\hat{L}^t$

In [ ]:
import numpy as np



# Excepciones
class slsException(Exception):
    def __init__(self,valor):
        self.valor = str(valor)
    def __str__(self):
        errores = {"0":"Las dimensiones son diferentes. La matriz debe ser cuadrada.",
                  "1":"Los elementos de la diagonal son cero. La matriz no es invertible.",
                  "2":"Los elementos de la diagonal son cero.",
                  "3":"Los elementos de la diagonal deben ser positivos."}
        if self.valor in errores:
            return f"Error {self.valor}. {errores[self.valor]}"
        else:
            return "Error no clasificado"
# Factorización de Cholesky LL^t
def chol_LLt(M):
    m,n = M.shape # dimensiones
    if m != n: # mismas dimenciones
        raise slsException(0)
    L = np.zeros(M.shape,dtype = "float64") # triangular inferior
    for k in range(n):
        s = 0.0
        for r in range(k):
            s += L[k,r]**2
        L[k,k] = np.sqrt(M[k,k] - s) # sacamos raices de la diagonal
        for j in range(k+1,n):
            s = 0.0
            for r in range(k):
                s += L[j,r]*L[k,r] # aplicamos la formula vista en clase
            L[j,k] = (M[j,k] - s)/L[k,k]
    return L
# Factorización LU
def flu(A):
    m,n = A.shape
    if m!= n:
        raise slsException(0)
    L,U = np.eye(n),np.zeros((n,n)) # triangulares
    for j in range(n):
        for k in range(j,n):
            s = 0
            for i in range(j): # j-1 = range(j)
                s += L[j,i]*U[i,k] # fila de A = (fila de L)(U)
            U[j,k] = A[j,k] - s # fila de U = fila de A
        for k in range(j+1,n):
            s = 0
            for i in range(j):
                s += L[k,i]*U[i,j]
            L[k,j] = (A[k,j] - s)/U[j,j] # aplicamos fórmula vista en clase 
    return L,U
# Factorización de Cholesky LDL^t
def chol_LDLt(M):
    m,n = M.shape
    if m != n:
        raise slsException(0)
    L,U = flu(M) # A = LU
    D = np.diag(np.diag(U)) # D = diagonal de U 
    return L,D



# Factorización de Cholesky LL^t (imprime la factorización (L) y la comprobación (A = LL^t))
def f_chol_LLt(A):
  L = chol_LLt(A)
  print(f"L = \n {L,np.allclose(L@L.T,A)}") # comprobación

# Factorización de Cholesky LDL^t (imprime la factorización (L y D) y la comprobación (A = LDL^t))
def f_chol_LDLt(A):
  L,D = chol_LDLt(A)
  print(f"L = \n {L}","\n",f"D = \n {D}",np.allclose(L@D@L.T,A)) # comprobación



# Implementación de los métodos creados:
# Matrices dadas
A = np.array([[2,-1,0],[-1,2,-1],[0,-1,2]])
B = np.array([[4,1,1,1],[1,3,-1,1],[1,-1,2,0],[1,1,0,2]])
C = np.array([[4,1,-1,0],[1,3,-1,0],[-1,-1,5,2],[0,0,2,4]])
E = np.array([[6,2,1,-1],[2,4,1,0],[1,1,4,-1],[-1,0,-1,3]])
# Factorización de Cholesky A = LL^t
print("FACTORIZACIÓN DE CHOLESKY A = LL^t")
print("\n Para la matriz A:")
f_chol_LLt(A)
print("\n Para la matriz B:")
f_chol_LLt(B)
print("\n Para la matriz C:")
f_chol_LLt(C)
print("\n Para la matriz E:")
f_chol_LLt(E)
# Factorización de Cholesky A = LDL^t
print("\n\n FACTORIZACIÓN DE CHOLESKY A = LDL^t")
print("\n Para la matriz A:")
f_chol_LDLt(A)
print("\n Para la matriz B:")
f_chol_LDLt(B)
print("\n Para la matriz C:")
f_chol_LDLt(C)
print("\n Para la matriz E:")
f_chol_LDLt(E)

FACTORIZACIÓN DE CHOLESKY A = LL^t

 Para la matriz A:
L = 
 (array([[ 1.41421356,  0.        ,  0.        ],
       [-0.70710678,  1.22474487,  0.        ],
       [ 0.        , -0.81649658,  1.15470054]]), True)

 Para la matriz B:
L = 
 (array([[ 2.        ,  0.        ,  0.        ,  0.        ],
       [ 0.5       ,  1.6583124 ,  0.        ,  0.        ],
       [ 0.5       , -0.75377836,  1.08711461,  0.        ],
       [ 0.5       ,  0.45226702,  0.0836242 ,  1.24034735]]), True)

 Para la matriz C:
L = 
 (array([[ 2.        ,  0.        ,  0.        ,  0.        ],
       [ 0.5       ,  1.6583124 ,  0.        ,  0.        ],
       [-0.5       , -0.45226702,  2.13200716,  0.        ],
       [ 0.        ,  0.        ,  0.93808315,  1.76635217]]), True)

 Para la matriz E:
L = 
 (array([[ 2.44948974,  0.        ,  0.        ,  0.        ],
       [ 0.81649658,  1.82574186,  0.        ,  0.        ],
       [ 0.40824829,  0.36514837,  1.92353841,  0.        ],
       [-0.4082482